In [ ]:
# Import Pipeline
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

# Import filter/screen
from quantopian.pipeline.filters import QTradableStocksUS

# Import data
from quantopian.pipeline.data import USEquityPricing
from quantopian.pipeline.data.factset import Fundamentals as ff
from quantopian.pipeline.data.morningstar import Fundamentals as mf

# Import classifier
from quantopian.pipeline.classifiers.fundamentals import Sector

# Import Alphalens
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet

In [ ]:
# Constants
start_date = '2007-1-1'
end_date = '2020-5-20'

factor_name = 'fcf_yld'

sector_labels, sector_labels[-1] = dict(Sector.SECTOR_NAMES), "Unknown"

group_neutral = True

long_short = True

In [ ]:
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters.morningstar import IsPrimaryShare
from quantopian.pipeline.data import morningstar

# Create pipeline function
def make_pipeline():
    
    # Get our primary data
    
    '''
    sector = Sector()
    
    fcf = mf.free_cash_flow.latest
    ev = mf.enterprise_value.latest
    
    # Calculate factor
    
    fcf_yld = fcf / ev
    
    #Create screens
    
    factor_screen = fcf_yld.notnull()
    
    sector_screen = sector.notnull()
    
    # Return pipeline
    
    return Pipeline(
        columns={
            factor_name: fcf_yld,
            'sector': sector
        },
        screen=QTradableStocksUS() & factor_screen & sector_screen,
    )
    '''
    
    # Filter for primary share equities. IsPrimaryShare is a built-in filter.
    primary_share = IsPrimaryShare()
 
    # Equities listed as common stock (as opposed to, say, preferred stock).
    # 'ST00000001' indicates common stock.
    common_stock = morningstar.share_class_reference.security_type.latest.eq(
        'ST00000001')
 
    # Non-depositary receipts. Recall that the ~ operator inverts filters,
    # turning Trues into Falses and vice versa
    not_depositary = ~morningstar.share_class_reference.is_depositary_receipt.latest
 
    # Equities not trading over-the-counter.
    not_otc = ~morningstar.share_class_reference.exchange_id.latest.startswith(
        'OTC')
 
    # Not when-issued equities.
    not_wi = ~morningstar.share_class_reference.symbol.latest.endswith('.WI')
 
    # Equities without LP in their name, .matches does a match using a regular
    # expression
    not_lp_name = ~morningstar.company_reference.standard_name.latest.matches(
        '.* L[. ]?P.?$')
 

    # Equities whose most recent Morningstar market cap is not null have
    # fundamental data and therefore are not ETFs.
    have_market_cap = morningstar.valuation.market_cap.latest.notnull()
 
    LowVar = 6
    HighVar = 40
 
    # Filter for stocks that pass all of our previous filters.
    tradeable_stocks = (
        primary_share
        & common_stock
        & not_depositary
        & not_otc
        & not_wi
        & not_lp_name
        & have_market_cap

    )
    ''' 
    base_universe = AverageDollarVolume(
        window_length=20,
        mask=tradeable_stocks
    ).percentile_between(LowVar, HighVar)
    '''
    sector = Sector()
    
    ShortAvg = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=3, 
        mask=tradeable_stocks
    )
 
    # Long close price average.
    LongAvg = SimpleMovingAverage(
        inputs=[USEquityPricing.close],
        window_length=45, 
        mask=tradeable_stocks
    )
 
    percent_difference = -(ShortAvg - LongAvg) / LongAvg
    
    factor_screen = percent_difference.notnull()
    
    sector_screen = sector.notnull()
    
    # Return pipeline
    
    return Pipeline(
        columns={
            factor_name: percent_difference,
            'sector': sector
        },
        screen=QTradableStocksUS() & factor_screen & sector_screen,
    )



In [ ]:
# Get factor data from pipeline
factor_data = run_pipeline(make_pipeline(), start_date, end_date)

In [ ]:
factor_data.head()

In [ ]:
# Get pricing data
pricing_data = get_pricing(factor_data.index.levels[1], start_date, end_date, fields='open_price')

In [ ]:
pricing_data.head()

In [ ]:
# Get factor and returns data
merged_data = get_clean_factor_and_forward_returns(
    factor=factor_data[factor_name],
    prices=pricing_data,
    periods=(1,5,10),
    quantiles=5,
    groupby=factor_data['sector'],
    groupby_labels = sector_labels,
    binning_by_group = group_neutral,
)

In [ ]:
merged_data.tail()

In [ ]:
# Create tear-sheet to analyse
create_full_tear_sheet(merged_data, long_short=long_short, by_group=True, group_neutral=group_neutral)

In [ ]:
1+1